In [1]:
import pandas as pd
import numpy as np
from models.dit import DiT
import torch
import os
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../UCRArchive_2018/NonInvasiveFetalECGThorax1/NonInvasiveFetalECGThorax1_TRAIN.tsv', sep='\t', index_col=False, header=None)
test = pd.read_csv('../UCRArchive_2018/NonInvasiveFetalECGThorax1/NonInvasiveFetalECGThorax1_TEST.tsv', sep='\t', index_col=None, header=None)

In [3]:
train = torch.Tensor(train.drop(columns=0).to_numpy()).unsqueeze(1)
test = torch.Tensor(test.drop(columns=0).to_numpy()).unsqueeze(1)

In [4]:
test.shape

torch.Size([1965, 1, 750])

In [5]:
net = DiT(input_size=750,
        patch_size=5,
        in_channels=1,
        hidden_size=300,
        depth=28,
        num_heads=10,
        mlp_ratio=4.0,
        class_dropout_prob=0.1,
        learn_sigma=False,)

In [6]:
x = torch.randn(4,1,750)
y = test[:4]
t = torch.randint(low=0, high=10000, size=(4,))